In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cat Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_data, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 5}
KNN Model Metrics:
Train Accuracy: 0.7668
Train Precision: 0.7554
Train Recall: 0.7668
Train F1 Score: 0.7554
Train Confusion Matrix:
[[ 12  12   8]
 [ 15  65  52]
 [  5  19 288]]

Test Accuracy: 0.6780
Test Precision: 0.6861
Test Recall: 0.6780
Test F1 Score: 0.6812
Test Confusion Matrix:
[[  3   4   6]
 [  2  17  22]
 [  5  27 119]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      76.68%
Test Accuracy       67.80%
Train-Test Gap      8.88%
ROC-AUC             0.63
Overfitting         No


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 10}
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9937
Train Rec

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.8256
Train Precision: 0.8318
Train Recall: 0.8256
Train F1 Score: 0.8061
Train Confusion Matrix:
[[  5  18   9]
 [  0  91  41]
 [  0  15 297]]

Test Accuracy: 0.7463
Test Precision: 0.7163
Test Recall: 0.7463
Test F1 Score: 0.7300
Test Confusion Matrix:
[[  0  12   1]
 [  0  21  20]
 [  0  19 132]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      82.56%
Test Accuracy       74.63%
Train-Test Gap      7.93%
ROC-AUC             0.71
Overfitting         No


Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 0.5, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7707
Test Precision: 0.7738
Test Recall: 0.7707
Test F1 Score: 0.7453
Test Confusion Matrix:
[[  1   7   5]
 [  0  18  23]
 [  0  12 139]]


Performance Table for XG BOOST:
Metric              Value     
------------------------------
Train Accuracy      99.37%
Test Accuracy       77.07%
Train-Test Gap      22.30%
ROC-AUC             0.68
Overfitting         Yes


Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7332
Train Precision: 0.7534
Train Recall: 0.7332
Train F1 Score: 0.7373
Train Confusion Matrix:
[[ 16  13   3]
 [  1  83  48]
 [  0  62 250]]

Test Accuracy: 0.6293
Test Precision: 0.7047
Test Recall: 0.6293
Test F1 Score: 0.6471
Test Confusion Matrix:
[[  1  10   2]
 [  0  24  17]
 [  2  45 104]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      73.32%
Test Accuracy       62.93%
Train-Test Gap      10.39%
ROC-AUC             0.65
Overfitting         Yes


Running CatBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [5]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_data, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1}
Logistic Regression Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9832
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 31   1   0]
 [  0 127   5]
 [  0   2 310]]

Test Accuracy: 0.7415
Test Precision: 0.7347
Test Recall: 0.7415
Test F1 Score: 0.7319
Test Confusion Matrix:
[[  2   7   4]
 [  1  19  21]
 [  1  19 131]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      98.32%
Test Accuracy       74.15%
Train-Test Gap      24.17%
ROC-AUC             0.68
Overfitting         Yes


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 1}
SVM Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.6780
Test Precision: 0.7044
Test Recall: 0.6780
Test F1 Score: 0.6895
Test Confusion Matrix:
[[  4   5   4]
 [  2  19  20]
 [ 11  24 116]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      99.37%
Test Accuracy       67.80%
Train-Test Gap      31.56%
ROC-AUC             0.66
Overfitting         Yes




In [6]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to BERT embeddings
def convert_to_bert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to generate BERT embeddings for a single text
    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embedding as the sentence representation
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Apply the BERT embedding generation function to the text column
    bert_embeddings = np.array([get_bert_embedding(text) for text in df[text_column].astype(str)])

    # Print the shape of the BERT embeddings
    print(f"BERT embeddings shape for '{text_column}': {bert_embeddings.shape}")

    # Return the BERT embeddings and the original DataFrame
    return pd.DataFrame(bert_embeddings), df

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for BERT embedding conversion
text_column = 'answer'

# Convert text column to BERT embeddings and get the original DataFrame
bert_data, original_data = convert_to_bert_embeddings(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(bert_data, variance_threshold=0.95)

# Adjust y to ensure labels start from 0 (if needed)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BERT embeddings shape for 'answer': (681, 768)
Number of components selected: 171
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 3}
KNN Model Metrics:
Train Accuracy: 0.9979
Train Precision: 0.9979
Train Recall: 0.9979
Train F1 Score: 0.9979
Train Confusion Matrix:
[[ 32   0   0]
 [  0 131   1]
 [  0   0 312]]

Test Accuracy: 0.6683
Test Precision: 0.6843
Test Recall: 0.6683
Test F1 Score: 0.6746
Test Confusion Matrix:
[[  2   7   4]
 [  4  16  21]
 [  3  29 119]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      99.79%
Test Accuracy       66.83%
Train-Test Gap      32.96%
ROC-AUC             0.64
Overfitting         Yes


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'sgd', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.7941
Train Precision: 0.7820
Train Recall: 0.7941
Train F1 Score: 0.7761
Train Confusion Matrix:
[[  6  15  11]
 [  1  81  50]
 [  2  19 291]]

Test Accuracy: 0.7756
Test Precision: 0.7813
Test Recall: 0.7756
Test F1 Score: 0.7674
Test Confusion Matrix:
[[  2   9   2]
 [  1  23  17]
 [  0  17 134]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      79.41%
Test Accuracy       77.56%
Train-Test Gap      1.85%
ROC-AUC             0.74
Overfitting         No


Running XGBoost model...
Best Params: {'subsample': 0.8, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 1.0000
Train Precision: 1.0000
Train Recall: 1.0000
Train F1 Score: 1.0000
Train Confusion Matrix:
[[ 32   0   0]
 [  0 132   0]
 [  0   0 312]]

Test Accuracy: 0.7415
Test Precision: 0.6717
Test Recall: 0.7415
Test F1 S

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7374
Train Precision: 0.7372
Train Recall: 0.7374
Train F1 Score: 0.7151
Train Confusion Matrix:
[[  7  16   9]
 [  0  60  72]
 [  0  28 284]]

Test Accuracy: 0.6732
Test Precision: 0.6275
Test Recall: 0.6732
Test F1 Score: 0.6493
Test Confusion Matrix:
[[  0   4   9]
 [  0  12  29]
 [  2  23 126]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      73.74%
Test Accuracy       67.32%
Train-Test Gap      6.42%
ROC-AUC             0.56
Overfitting         No


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.7311
Train Precision: 0.6699
Train Recall: 0.7311
Train F1 Score: 0.6964
Train Confusion Matrix:
[[  0  21  11]
 [  0  64  68]
 [  0  28 284]]

Test Accuracy: 0.7463
Test Precision: 0.7047
Test Recall: 0.7463
Test F1 Score: 0.7248
Test Confusion Matrix:
[[  0   7   6]
 [  0  21  20]
 [  0  19 132]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      73.11%
Test Accuracy       74.63%
Train-Test Gap      1.52%
ROC-AUC             0.68
Overfitting         No


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7220
Test Precision: 0.7052
Test Recall: 0.7220
Test F1 Score: 0.7103
Test Confusion Matrix:
[[  1   8   4]
 [  2  17  22]
 [  1  20 130]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      99.37%
Test Accuracy       72.20%
Train-Test Gap      27.17%
ROC-AUC             0.67
Overfitting         Yes




In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from lime.lime_text import LimeTextExplainer
from transformers import BertTokenizer, BertModel
from google.colab import drive
import torch
import numpy as np

# Load data from an Excel file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Preprocess data: Use only the ANSWER column and map labels to "not strong" and "strong"
def preprocess_data(data, label_column='confidence', answer_column='answer'):
    # Map labels to "not strong" and "strong"
    label_map = {-1: 'not strong', 0: 'not strong', 1: 'strong'}
    data[label_column] = data[label_column].map(label_map)

    # Use only the ANSWER column for input features
    data['TEXT'] = data[answer_column]
    data['TEXT'] = data['TEXT'].fillna('')

    return data

# Create BERT embeddings for the dataset
def create_bert_embedding_matrix(data, text_column='TEXT', model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    embeddings = []
    for text in data[text_column]:
        # Tokenize the text and convert to input IDs
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=128)

        # Pass through BERT model to get embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Use the CLS token's embedding (first token) as the sentence embedding
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)

    return np.array(embeddings)

# Train AdaBoost classifier
def train_adaboost(X_train, y_train, n_estimators=150, learning_rate=1.0):
    # Define AdaBoostClassifier model
    adaboost = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate)
    adaboost.fit(X_train, y_train)
    return adaboost

# Generate LIME explanations for a sample text
def lime_explain(model, text, label_classes, tokenizer, bert_model):
    explainer = LimeTextExplainer(class_names=label_classes)

    def predict_proba(texts):
        embeddings = []
        for t in texts:
            inputs = tokenizer(t, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
            with torch.no_grad():
                outputs = bert_model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
            embeddings.append(cls_embedding)
        return model.predict_proba(np.array(embeddings))

    exp = explainer.explain_instance(text, predict_proba)
    return exp

# Main function
def main():
    # Mount Google Drive to access the dataset
    drive.mount('/content/drive')

    # Define file paths
    data_file_path = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

    # Load data
    data = load_data(data_file_path)

    # Preprocess data
    data = preprocess_data(data)

    # Load BERT tokenizer and model
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name)

    # Create embedding matrix using BERT
    X = create_bert_embedding_matrix(data, model_name=model_name)
    y = data['confidence']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train AdaBoost classifier
    adaboost_model = train_adaboost(X_train, y_train)

    # Predict and evaluate
    y_pred = adaboost_model.predict(X_test)

    # Overall Metrics
    accuracy_score_value = accuracy_score(y_test, y_pred)
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_f1_score = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print()
    print(f"Accuracy Score: {accuracy_score_value:.2f}")
    print(f"Precision: {weighted_precision:.2f}")
    print(f"Recall: {weighted_recall:.2f}")
    print(f"F1 Score: {weighted_f1_score:.2f}")

    # Built-in confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=['not strong', 'strong'])
    print("Confusion Matrix:\n", cm)

    # LIME explanation for a sample text
    sample_index = 237  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(adaboost_model, sample_text, label_classes=['not strong', 'strong'], tokenizer=tokenizer, bert_model=bert_model)
    explanation.show_in_notebook()

    # LIME explanation for another sample text
    sample_index = 6  # You can change this index to another sample index
    sample_text = data['answer'].iloc[sample_index]  # Access the answer based on the index
    explanation = lime_explain(adaboost_model, sample_text, label_classes=['not strong', 'strong'], tokenizer=tokenizer, bert_model=bert_model)
    explanation.show_in_notebook()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'lime'

In [2]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=6d2e588e71688ecb8bca714a526d7e955f557a211c64d86d3cb2002740a4f226
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime
